In [2]:
import sys
sys.path.append("utils/")
import wv_util as wv
from utils.dataset_utils import *

In [3]:
coords1, chips1, classes1 = wv.get_labels('/data/zjc4/xView_train.geojson')

100%|██████████| 601937/601937 [00:03<00:00, 164939.74it/s]


In [4]:
idxs = np.arange(0,coords1.shape[0]).reshape(-1,1)
coords1_idx = np.hstack( (coords1, idxs ))


In [62]:
coords1_idx.shape

(601937, 5)

In [63]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import tqdm
import pickle
import itertools
import cv2 as cv
import glob
import random
import os
import imp

from PIL import Image
from sklearn.model_selection import train_test_split
import dataset_utils as du

imp.reload(wv)
imp.reload(du)

import random
np.random.seed(20)
random.seed(20)
unique_chips = np.unique(chips1)
selected_images = np.random.choice(unique_chips,10)
#selected_images = np.random.choice(unique_chips,10)
idxs = np.isin(chips1,selected_images)

# Input desired classes
grouped_classes = [[77,73],[11,12],[13],[17,18,20,21],
       [19,23,24,25,28,29,60,61,65,26],[41,42,50,40,44,45,47,49]]

labels = ["building and facility" ,"small aircraft", 
          "large aircraft","vehicles","bus","boat"]

xdataset = du.XviewDataset(grouped_classes,labels, coords1_idx[idxs],chips1[idxs],classes1[idxs])
tif_names,distrbs = xdataset.splitTrainTest(debug=True)
dfs,gc = distrbs

string_sets = ["train","valid"]
datum = list(zip(string_sets,tif_names))

"""
Generate darknet images for 30 cm
"""
# Deleting all old images
os.system("rm /data/zjc4/chipped-30/data/images/*")
os.system("rm /data/zjc4/chipped-30/data/labels/*")
os.system("rm /data/zjc4/chipped-30/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-30/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-30/data/xview_img_test.txt")

dnf = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-30/data/",
                       input_dir_="/data/zjc4/",
                     coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf.transformDatum(datum,30,showImg=False,chipImage=True)

10it [00:00, 680.56it/s]


/data/zjc4/chipped-30/data/xview_img_train.txt
/data/zjc4/chipped-30/data/xview_img_valid.txt


In [64]:
import dataset_utils as du
imp.reload(du)

"""
Generate darknet images for 30 cm
"""
# Deleting all old images
os.system("rm /data/zjc4/chipped-90/data/images/*")
os.system("rm /data/zjc4/chipped-90/data/labels/*")
os.system("rm /data/zjc4/chipped-90/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-90/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-90/data/xview_img_test.txt")

dnf2 = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-90/data/",
                       input_dir_="/data/zjc4/",
                       coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf2.transformDatum(datum,90,showImg=False,chipImage=True)

/data/zjc4/chipped-90/data/xview_img_train.txt
/data/zjc4/chipped-90/data/xview_img_valid.txt


In [32]:
temp_df = dfs[1]
temp_df['count'] = pd.to_numeric(temp_df['count'])
temp_df[['grouped cls','count']].groupby(['grouped cls']).sum()

,count
grouped cls,
boat,0.0
building and facility,303.0
bus,104.0
large aircraft,0.0
small aircraft,0.0
vehicles,104.0
